## 4. Model Design and Implementation

4.1 Design a neural network model with only a flatten layer and two dense layers for classifying lung nodules. You may experiment with different sizes for the hidden layers to improve the training results.

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

train_chunks = train_chunks.clone().detach() if isinstance(train_chunks, torch.Tensor) else torch.tensor(train_chunks, dtype=torch.float32)
train_labels = train_labels.clone().detach() if isinstance(train_labels, torch.Tensor) else torch.tensor(train_labels, dtype=torch.long)
test_chunks = test_chunks.clone().detach() if isinstance(test_chunks, torch.Tensor) else torch.tensor(test_chunks, dtype=torch.float32)
test_labels = test_labels.clone().detach() if isinstance(test_labels, torch.Tensor) else torch.tensor(test_labels, dtype=torch.long)


train_dataset = TensorDataset(train_chunks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(test_chunks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


print(f"Train loader batch shape: {next(iter(train_loader))[0].shape}")
print(f"Test loader batch shape: {next(iter(test_loader))[0].shape}")


Train loader batch shape: torch.Size([32, 32, 48, 48])
Test loader batch shape: torch.Size([32, 32, 48, 48])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size_1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size_2, output_size)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


In [ ]:
input_size = 73728
hidden_size_1 = 128
hidden_size_2 = 64
output_size = 2

model = SimpleNN(input_size, hidden_size_1, hidden_size_2, output_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


SimpleNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=73728, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

4.2 Create an object to represent the loss function.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
print(loss_fn)




CrossEntropyLoss()


4.3 Create an object to represent the optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.0005
    lr: 0.0005
    maximize: False
    weight_decay: 0
)


4.4 Create a function to represent the training loop.

4.5 Execute the training loop. Display the change of training loss during the training process. Choose a reasonable value for the number of training epochs based on your observations.


In [ ]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)


        out = model(img)


        loss = loss_fn(out, label)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    scheduler.step()

    print(f"Epoch {epoch+1}/{n_epochs}, Training Loss: {running_loss/len(train_loader):.4f}")


    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for img, label in test_loader:
            img, label = img.to(device), label.to(device)

            out = model(img)

            loss = loss_fn(out, label)
            test_loss += loss.item()


            _, predicted = torch.max(out, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{n_epochs}, Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/20, Training Loss: 0.0007
Epoch 1/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 2/20, Training Loss: 0.0007
Epoch 2/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 3/20, Training Loss: 0.0007
Epoch 3/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 4/20, Training Loss: 0.0007
Epoch 4/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 5/20, Training Loss: 0.0007
Epoch 5/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 6/20, Training Loss: 0.0007
Epoch 6/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 7/20, Training Loss: 0.0007
Epoch 7/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 8/20, Training Loss: 0.0007
Epoch 8/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 9/20, Training Loss: 0.0007
Epoch 9/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 10/20, Training Loss: 0.0007
Epoch 10/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 11/20, Training Loss: 0.0007
Epoch 11/20, Test Loss: 1.6853, Test Accuracy: 99.90%
Epoch 12/20, Training Loss: 0.0007
Epoc